# Building database for node.js

In [3]:
!pwd

/Users/ryan/DH/web/toolbox_node/db


In [4]:
MODEL_ID='COHA_byhalfcentury_nonf_smpl'
MODEL_FN='../static/data/db/models/COHA_byhalfcentury_nonf/chained_combined/1800-1999.min=500.run=01.txt'

In [5]:
from scipy.spatial.distance import pdist,squareform

In [6]:
import gensim
M = gensim.models.KeyedVectors.load_word2vec_format(MODEL_FN)

In [7]:
M.init_sims()

In [8]:
words=list(M.vocab)

In [9]:
vecs=M.vectors_norm

In [10]:
distsX = squareform(pdist(vecs,metric='cosine'))

In [11]:
import pandas as pd

In [12]:
dfdist = pd.DataFrame(distsX,columns=words,index=words)

In [13]:
#list(dfdist['value_1800'].sort_values()[:100].index)

In [30]:
#M['virtue_1800']

In [14]:
n_top=50

In [15]:
# for word in words:
#     rowd = dict(dfdist.loc[word].sort_values()[:n_top+1])
#     del rowd[word]
#     #print(rowd, len(rowd))
#     #break
    
#     for i,(k,v) in enumerate(rowd.items()):
#         G.add_edge(word,k,sim_cos=1-v,sim_rank=i+1)

In [17]:
from tqdm import tqdm

import networkx as nx
G=nx.Graph()

for word in tqdm(words):
    row = dfdist.loc[word].sort_values().iloc[1:n_top+1]
    
    sim_rank=0
    for word2,result in zip(row.index,row):
        sim_rank+=1
        
        #if word2<=word: continue
        
        G.add_edge(word,word2,weight=1-result,sim_rank=sim_rank)
        #print(index,result)
    #break

100%|██████████| 31228/31228 [02:41<00:00, 193.17it/s]


In [19]:
G.size(),G.order()

(1117828, 31228)

In [20]:
#dict(row.sort_values()[:100])

In [21]:
#list(G.neighbors('value_1800'))

In [22]:
#nx.shortest_path(G,'value_1800','computer_1950')

In [23]:
import os
if not os.path.exists(MODEL_ID): os.mkdir(MODEL_ID)

In [24]:
ofn=MODEL_ID+'/data.simnet.csv'
with open(ofn,'w') as of:
    of.write('source,target,weight,sim_rank\n')
    for a,b,d in G.edges(data=True):
        of.write(a+','+b+','+str(d['weight'])+','+str(d['sim_rank'])+'\n')

### Save vecs too

In [25]:
from pymongo import MongoClient
client = MongoClient()

In [31]:
vecdb = client['vecdb'][MODEL_ID]

In [34]:
for word in tqdm(words):
    wvecs = [float(x) for x in M[word]]
    doc = {'word':word, 'vecs':wvecs}
    vecdb.insert(doc)

  0%|          | 0/31228 [00:00<?, ?it/s]/Users/ryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  after removing the cwd from sys.path.
100%|██████████| 31228/31228 [00:19<00:00, 1562.50it/s]
